In [14]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [18]:
# globalne promenljive

fs = 44100
f0 = 10_000
df = 1_000
dt = .1
small_segment = 100

In [20]:
# Funkcije koje koristimo

def GenerateRandom(length = 1000):
    bin_string = ""
    
    for _ in range(length):
        bin_string += '1' if bool(random.getrandbits(1)) else '0'
    
    return bin_string

def GenerateSignal(bin_code : str):

    # učitavanje globalnih promenljiva
    global fs, f0, df, dt

    # pravljenje frekvencija za pakete
    f = [f0 - df // 2, f0 + df // 2]

    # priprema za pravljenje mfsk niza
    samples_per_symbol = int(int(fs) * dt)
    
    # pravljenje mfsk niza sa frekvencijama
    mfsk = []
    for bit in bin_code:
        mfsk += [f[int(bit)] for _ in range(samples_per_symbol)]
    mfsk = np.array(mfsk)
    
    # pravljenje cpmfsk signala
    delta_phi = mfsk * np.pi / (fs / 2.0)
    phi = np.cumsum(delta_phi)
    cpmfsk = np.sin(phi)

    return cpmfsk

def AddingNoise(sig, snr):

    # pravljenje šuma sa određenim SNRom
    sig_watts = np.mean(sig ** 2)
    sig_db = 10 * np.log10(sig_watts)
    noise_db = sig_db - snr
    noise_watts = 10 ** (noise_db / 10)
    noise = np.random.normal(0, np.sqrt(noise_watts), len(sig))
    
    # spajanje signala i šuma
    signal_with_noise = sig + noise

    return signal_with_noise

def DemodulateSignal(signal, f0, f1, small_segment, fs):
    ...

def BER(input_bin, output_bin, snr):
    ...

In [ ]:
BERs = []
for noise in np.arange(1, -10, -.1):
    for _ in range(1000):
        input_bin = GenerateRandom(8 * 1000) # generisanje poruke
        signal = AddingNoise(GenerateSignal(input_bin), noise) # modulacija signala
        
        output_bin = DemodulateSignal(signal, f0 - df // 2, f0 + df // 2, small_segment, fs) # demodulacija
        ber_snr = BER(input_bin, output_bin, noise) # računanje BER/SNR
        BERs.append(ber_snr)

# plotovanje
fig, ax = plt.subplots(1, 1, figsize = (8, 2))
ax.plot(BERs)